# Librairies

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pylab as plt

# Load data

In [2]:
gdf = gpd.read_file('raw/Trafikmätning_för_biltrafik_mätplatser.geojson')

In [3]:
gdf.columns

Index(['pg_id', 'pg_nummer', 'gatunamn', 'delsträcka', 'rikt_1', 'rikt_2',
       'punkt_uppdaterat', 'year', 'mvd', 'max_fm', 'max_em', 'andel_tung',
       'medelhast', 'stats_uppdaterat', 'timestamp', 'ogc_fid', 'geometry'],
      dtype='object')

In [4]:
# translation to english
attributes_translation = [
    "pg_id",                # Unique identifier for each measurement location
    "pg_number",            # Identifier for the measurement point's position
    "street_name",          # Street where the measurement was conducted
    "segment",              # Position where the measurement was conducted
    "direction_1",          # Traffic direction included in the measurement
    "direction_2",          # Other traffic direction included in the measurement
    "point_updated",        # Last update of the measurement point information and/or geometry
    "year",                 # Measurement year
    "aadt",                 # Average daily traffic. Average traffic during everyday life for a given period of time 
                            # for all motor vehicles in both directions. The average is calculated based on the measurements 
                            # for the days Tuesday-To Thursday, weekend and commendation-free days.
    "max_am",               # Motor vehicles/hour during the morning peak hour
    "max_pm",               # Motor vehicles/hour during the afternoon peak hour
    "heavy_share",          # Motor vehicles over 3.5 tons as a percentage of total traffic, Applies to traffic in both directions.
    "avg_speed",            # Average speed of total traffic,  Applies to traffic in both directions.
    "stats_updated",        # Last update of statistical values; could be adjustments to previously published figures or new data
    "timestamp" ,            # Last update date of the traffic measurement objects
    "ogc_fid",
    "geometry",             # Type: MultiLineString, CRS: WGS84

]


In [5]:
gdf.columns = attributes_translation

In [11]:
gdf.year.value_counts().sort_index()[-10:]

year
2015    166
2016    166
2017    207
2018    223
2019    247
2020     81
2021    218
2022    208
2023    221
2024    228
Name: count, dtype: int64

In [15]:
gdf[['pg_id', 'year']].duplicated().sum()

0

In [23]:
# Every measure seems to be both direction
gdf.direction_1.isna().sum() == gdf.direction_2.isna().sum() == 0

True

# Map matching with OSM

In [25]:
# reduce the number of points to compute
gdf_unique = gdf[gdf.year >=2015].drop_duplicates(subset = ['pg_id'])

In [26]:
# The Multilinestring geometries are "sections" i.e. they are perpendicular to the road they monitor
# If we want to avoid creating linestrings with the wrong direction, we could still use the point matching technique taking the centroid
gdf_unique['geometry'] = gdf_unique.geometry.apply(lambda x : x.centroid)

In [27]:
import sys
from pathlib import Path

# Add the ../assets directory to sys.path
sys.path.append(str(Path("../../assets").resolve()))

from map_matching_OSM import points_matching

In [28]:
gdf_unique.shape

(759, 17)

In [29]:
# 17min
gdf_unique = points_matching(gdf_unique)

City downloaded


 18%|█▊        | 137/759 [00:00<00:02, 212.18it/s]

Value Error - No roads found nearby current index


 47%|████▋     | 355/759 [00:01<00:02, 200.65it/s]

Value Error - No roads found nearby current index


...

Value Error - No roads found nearby current index
Value Error - No roads found nearby current index


 97%|█████████▋| 735/759 [00:03<00:00, 224.63it/s]

Value Error - No roads found nearby current index
Value Error - No roads found nearby current index
Value Error - No roads found nearby current index
Value Error - No roads found nearby current index


100%|██████████| 759/759 [00:03<00:00, 216.11it/s]

We failed to match 9 sensors
...on a total of 759 sensors


In [30]:
gdf_unique[['street_name', 'osm_name']].drop_duplicates().dropna()

,street_name,osm_name
0,STORMGATAN,Stormgatan
1,SCHOUGENS BRO,Schougens bro
2,ÖSTERLENVÄGEN,Österlenvägen
3,SALLERUPSVÄGEN,Malmövägen
4,KLAGSTORPSVÄGEN,Klagstorpsvägen
...,...,...
1837,KALKSTENSVÄGEN,Kalkstensvägen
1874,SIMRISBANVÄGEN,Simrisbanvägen
1887,NORRA SKOLGATAN,Norra Skolgatan
1941,ÖSTRA FÄLADSGATAN,Östra Fäladsgatan


In [31]:
# Add OSM attributes to main dataframe
gdf = gdf[gdf.year >=2015].set_index('pg_id').drop('geometry', axis = 1).join(
    gdf_unique.set_index('pg_id')[['osm_name', 'osm_type', 'osm_lanes', 'osm_oneway', 'osm_distance', 'osmid', 'geometry']],
    how = 'inner'
)

In [32]:
gdf[gdf.year >=2015].shape

(1965, 22)

In [33]:
gdf.columns

Index(['pg_number', 'street_name', 'segment', 'direction_1', 'direction_2',
       'point_updated', 'year', 'aadt', 'max_am', 'max_pm', 'heavy_share',
       'avg_speed', 'stats_updated', 'timestamp', 'ogc_fid', 'osm_name',
       'osm_type', 'osm_lanes', 'osm_oneway', 'osm_distance', 'osmid',
       'geometry'],
      dtype='object')

In [34]:
gdf.rename(columns = 
           {
               'street_name':'raw_name',
               'aadt':'AADT',
               'heavy_share':'TR_pct_AADT'
           }, inplace=True)

In [35]:
gdf = gpd.GeoDataFrame(
    gdf, 
    geometry = 'geometry',
    crs = 'epsg:4326'
)

In [36]:
gdf['raw_oneway'] = False

# Final saving

In [37]:
# Saving
for year in range(2015, 2025): # gdf.year.unique():
    # Extract corresponding year
    sub = gdf[gdf.year == year]
    print(year, sub.shape[0])
    sub[[
        'AADT', 'TR_pct_AADT', 'geometry', 
        'raw_name', 'raw_oneway', 'osm_name', 'osm_type', 'osm_lanes', 'osm_oneway', 'osm_distance', 'osmid'
        ]].to_file(f'treated/Malmo_AADT_{year}.geojson', index=False)

2015 166
2016 166
2017 207
2018 223
2019 247
2020 81
2021 218
2022 208
2023 221
2024 228
